In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Bring biomass data into dataframe

In [2]:
Observations = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='Observations')
obsPhenology = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='obsPhenology')
ObsStructure = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='ObsStructure')
Obs2016 = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='Obs2016')
Obs2015 = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='Obs2015')
RS_SW_Obs = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='RS_SW_Obs')
RS_Oats_9899_leaf = pd.read_excel('C:\Users\cflhxb\Dropbox\APSIMOats\Observed.xlsx',sheetname='RS_Oats_9899_leaf')

In [3]:
AllDataTabs = [Observations,obsPhenology,ObsStructure,Obs2016,Obs2015,RS_SW_Obs,RS_Oats_9899_leaf]

## Make a column index

In [4]:
ColumnIndex = Observations.columns
for Table in AllDataTabs:
    ColumnIndex = ColumnIndex  | Table.columns  # the union of the merged data frame and the next in the list

## Make row index

In [5]:
RowIndexData = Observations.ix[:,Observations.columns.isin(['SimulationName','Clock.Today'])]
for Table in AllDataTabs:
    RowIndexData.append(Table.ix[:,Table.columns.isin(['SimulationName','Clock.Today'])])
RowIndex = RowIndexData.drop_duplicates()
RowIndex.set_index(['SimulationName','Clock.Today'],inplace=True)

## Make empty Data frame

In [6]:
ObsDataFrame = pd.DataFrame(index=RowIndex.index, columns=ColumnIndex)

## Set indexes in the smaller tables and deal to duplicate rows

In [7]:
from collections import Counter
def MakeMean(ser):
    Series = pd.Series(index=ser.columns)
    for Group in ser.columns:
        if isinstance(Group, float):
            Series[Group] = ser[Group].mean()
        else:
            Series[Group] = Counter(ser[Group]).most_common(1)[0][0]
    return Series        

In [8]:
CompressedData = []
for Table in AllDataTabs:
    Table.set_index(['SimulationName','Clock.Today'],inplace=True)
    Compressed = Table.groupby(level=['SimulationName','Clock.Today']).apply(MakeMean)
    CompressedData.append(Compressed)

In [9]:
for DataTable in range(7):
    ObsDataFrame.update(CompressedData[DataTable])


In [10]:
ObsDataFrame

AllansCut Clock.Today Fert  \
SimulationName            Clock.Today                              
RS_Oats_9899_IrrigED1     1998-11-04        NaN         NaN  NaN   
                          1998-11-11        NaN         NaN  NaN   
                          1998-11-17        NaN         NaN  NaN   
                          1998-11-23        NaN         NaN  NaN   
                          1998-12-02        NaN         NaN  NaN   
                          1998-12-09        NaN         NaN  NaN   
                          1998-12-16        NaN         NaN  NaN   
                          1998-12-24        NaN         NaN  NaN   
                          1998-12-30        NaN         NaN  NaN   
                          1999-01-07        NaN         NaN  NaN   
                          1999-01-12        NaN         NaN  NaN   
                          1999-01-19        NaN         NaN  NaN   
                          1999-01-28        NaN         NaN  NaN   
                          1999-02-03        NaN         NaN  NaN   
                          1999-02-10        NaN         NaN  NaN   
                          1999-02-18        NaN         NaN  NaN   
                          1999-02-24        NaN         NaN  NaN   
                          1999-03-03        NaN         NaN  NaN   
RS_Oats_9899_IrrigED2     1998-11-04        NaN         NaN  NaN   
                          1998-11-11        NaN         NaN  NaN   
                          1998-11-17        NaN         NaN  NaN   
                          1998-11-23        NaN         NaN  NaN   
                          1998-12-02        NaN         NaN  NaN   
                          1998-12-09        NaN         NaN  NaN   
                          1998-12-16        NaN         NaN  NaN   
                          1998-12-24        NaN         NaN  NaN   
                          1998-12-30        NaN         NaN  NaN   
                          1999-01-07        NaN         NaN  NaN   
                          1999-01-12        NaN         NaN  NaN   
                          1999-01-19        NaN         NaN  NaN   
...                                         ...         ...  ...   
Roma2006CvCoolibahCutTwo  2006-08-10        Two         NaN  NaN   
                          2006-09-03        Two         NaN  NaN   
                          2006-09-04        Two         NaN  NaN   
                          2006-09-19        Two         NaN  NaN   
                          2006-10-18        Two         NaN  NaN   
Roma2006CvCoolibahCutOne  2006-07-18        One         NaN  NaN   
                          2006-08-09        One         NaN  NaN   
                          2006-08-10        One         NaN  NaN   
                          2006-09-04        One         NaN  NaN   
                          2006-10-18        One         NaN  NaN   
Roma2006CvCoolibahCutNone 2006-07-18       None         NaN  NaN   
                          2006-08-10       None         NaN  NaN   
                          2006-09-04       None         NaN  NaN   
                          2006-10-18       None         NaN  NaN   
Roma2006CvTaipanCutTwo    2006-07-18        Two         NaN  NaN   
                          2006-08-09        Two         NaN  NaN   
                          2006-08-10        Two         NaN  NaN   
                          2006-09-03        Two         NaN  NaN   
                          2006-09-04        Two         NaN  NaN   
                          2006-09-19        Two         NaN  NaN   
                          2006-10-18        Two         NaN  NaN   
Roma2006CvTaipanCutOne    2006-07-18        One         NaN  NaN   
                          2006-08-09        One         NaN  NaN   
                          2006-08-10        One         NaN  NaN   
                          2006-09-04        One         NaN  NaN   
                          2006-10-18        One         NaN  NaN   
Roma2006CvTaipanCutNone   

In [11]:
ObsDataFrame.to_excel('C:\APSIMX\Prototypes\Oats\Observed.xlsx',merge_cells=False,sheet_name='Observed')